# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [30]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [48]:
def date_sorter():
    """Sort notes in dates.txt by chronological order of dates."""

    # Columns for storing the dates
    date_cols = ['Year', 'Month', 'Day']

    # Months dictionary
    months_dic = {
        'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06',
        'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'
    }

    # Regex patterns for extracting dates
    days = '|'.join([str(day) for day in range(1, 32)] + ['0' + str(day) for day in range(1, 10)])
    months = '|'.join(list(months_dic.values()) + [str(month) for month in range(1, 10)])
    regex_day = r'(?P<Day>' + days + r')'
    regex_month_1 = r'(?P<Month>' + '|'.join(list(months_dic.keys())) + r')'
    regex_month_2 = r'(?P<Month>' + months + r')'
    regex_year_1 = r'(?P<Year>(?<![0-9])(?:(?:[789]{1}[0-9]{1})|(?:(?:19|20){1}[0-9]{2}))(?![0-9]))'
    regex_year_2 = r'(?P<Year>(?<![0-9])(?:(?:19|20){1}[0-9]{2})(?![0-9]))'
    regexs = [
        r'(?P<Date>' + regex_day + r' +' + regex_month_1 + r'[a-z.,]* +' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_1 + r'[a-z.,]*[ -]+' + regex_day + r'[a-z,]*[ -]+' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_2 + r'[-/]+' + regex_day + r'[-/]+' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_1 + r'[a-z.,]*[ -]+(?:' + regex_day + r'[a-z,]*[ -]+)?' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_2 + r'[-/]+(?:' + regex_day + r'[-/]+)?' + regex_year_1 + r')',
        r'(?P<Date>(?:' + regex_month_2 + r'[-/]+(?:' + regex_day + r'[-/]+)?)?' + regex_year_2 + r')'
    ]

    remain_notes = df.copy()
    dates_dfs = []

    for regex in regexs:
        # Extract dates using regex
        dates = remain_notes.str.lower().str.extract(regex)

        # Fill missing Month and Day values
        dates[date_cols[1:]] = dates[date_cols[1:]].fillna("1")

        # Remove rows with NaN values
        dates = dates.dropna()

        # Map month names to numbers
        dates['Month'] = dates['Month'].apply(lambda m: months_dic[m] if m in months_dic else m)

        # Convert columns to integers
        dates[date_cols] = dates[date_cols].astype(int)

        # Fix 2-digit years
        dates['Year'] = dates['Year'].apply(lambda y: y + 1900 if y < 100 else y)

        # Collect processed dates
        dates_dfs.append(dates)

        # Remove processed notes
        remain_notes = remain_notes.drop(dates.index.values)

    # Combine and sort dates
    sorted_dates = pd.concat(dates_dfs).sort_values(date_cols)

    # Return indices of notes in chronological order
    return pd.Series(data=sorted_dates.index.values, dtype=int)

date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64